In [598]:
import collections
from datetime import datetime as dt
import itertools
import math
import more_itertools
import networkx as nx
import numpy as np
from PIL import Image
import random
import re
from skimage.transform import rescale
from skimage import measure as sk_measure
import time


def get_input(day, split=None, f=str.strip):
    fin = f'dat/2024-{day:02}.txt'
    if split is None:
        input = open(fin, 'r').readlines()
    else:
        input = open(fin, 'r').read().split(split)
    
    if f is not None:
        input = [f(x) for x in input]
    
    return input

def timestamp(ts):
    date = dt.fromtimestamp(ts)
    return f'[{date.hour:02}:{date.minute:02}:{date.second:02}]'

def output(*args, ts=0):
    out = ' '.join([str(x) for x in args])
    if ts:
        ts = timestamp(ts)
        sz = 76 - len(out) - len(ts)
        if sz > 0:
            out += ' ' * sz
        out += ts
    print(out)

def as_ints(input):    
    nums = []
    for x in input:
        x = [int(c) for c in x]
        nums.append(np.asarray(x))
    return np.asarray(nums)

UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
clockwise = {
    UP: RIGHT,
    RIGHT: DOWN,
    DOWN: LEFT,
    LEFT: UP
}
anticlockwise = {
    UP: LEFT,
    LEFT: DOWN,
    DOWN: RIGHT,
    RIGHT: UP
}
def get_adjacent(x, y):
    pts = []
    for d in (UP, DOWN, LEFT, RIGHT):
        pts.append((x+d[1], y+d[0]))
    return pts

# Day 1

In [205]:
input = get_input(1)

def day_one(a, two=False):
    left = []
    right = []
    for row in a:
        l, r = row.split()
        left.append(int(l))
        right.append(int(r))
        
    left.sort()
    right.sort()
    out = 0
    if two:
        for i in left:
            out += i * right.count(i)
    else:
        for l, r in zip(left, right):
            out += abs(l - r)
    
    return out
        
    

output(day_one(input), ts=1733259903)
output(day_one(input, True), ts=1733260045)

3508942                                                           [16:05:03]
26593248                                                          [16:07:25]


# Day 2

In [204]:
input = """7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
4 3 6 7 9""".split('\n')
input = get_input(2)

def asc_or_desc(a, b, asc):
    return (a < b) == asc

def safe_dist(a, b):
    return 0 < abs(a-b) < 4

def is_safe(level):
    asc = None
    for i in range(len(level)-1):
        if asc is None:
            asc = level[i] < level[i+1]
            #print(level, asc)
        else:
            if not asc_or_desc(level[i], level[i+1], asc):
                #print(level[i],level[i+1],asc)
                break
        if not safe_dist(level[i], level[i+1]):
            #print(level[i],level[i+1],abs(level[i]-level[i+1]))
            break
    else:
        #print('SAFE', row)
        return True
    return False
        
def day_two(a, two=False):
    num_safe = 0
    for row in a:
        level = [int(i) for i in row.split()]
        if two:
            if is_safe(level):
                num_safe += 1
            else:
                for i in range(len(level)):
                    sublevel = [level[j] for j in range(len(level)) if j != i]
                    if is_safe(sublevel):
                        num_safe += 1
                        break
                
        else:
            if is_safe(level):
                num_safe += 1
    
    return num_safe
        

output(day_two(input), ts=1733261074)
output(day_two(input, True), ts=1733263415)

236                                                               [16:24:34]
308                                                               [17:03:35]


# Day 3

In [203]:
input = """xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)
+mul(32,64]then(mul(11,8)mul(8,5))""".split('\n')
input = """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))""".split('\n')
input = get_input(3)

def mul(x,y):
    return x*y

def get_enabled(s, idx, exp, enabled):
    regex_do = re.compile('do()')
    regex_dont = re.compile("don't()")
    
    if enabled:
        last_do = -1
        last_dont = -2
    else:
        last_do = -2
        last_dont = -1

    if exp:
        end = exp.start()
    else:
        end = len(s)
    
    pos = idx
    while True:
        exp = regex_do.search(s, pos, end)
        if not exp:
            break
        pos = exp.end()
        last_do = exp.start()
    
    pos = idx
    while True:
        exp = regex_dont.search(s, pos, end)
        if not exp:
            break
        pos = exp.end()
        last_dont = exp.start()
    
    return last_do > last_dont

def find_total(s, enabled=True, two=False):
    regex = re.compile('mul[(][0-9]{1,3},[0-9]{1,3}[)]')
    idx = 0
    total = 0
    while True:
        exp = regex.search(s, idx)
        enabled = get_enabled(s, idx, exp, enabled)
        if not exp:
            break
        idx = exp.end()
        #print(exp, enabled)
        if enabled or not two:
            total += eval(s[exp.start():exp.end()])
    return total, enabled

def day_three(a, two=False):
    total = 0
    enabled = True
    for row in a:
        subtotal, enabled = find_total(row, enabled, two)
        total += subtotal
    return total

output(day_three(input), ts=1733266367) #34873487 low
output(day_three(input, True), ts=1733267751)

189600467                                                         [17:52:47]
107069718                                                         [18:15:51]


# Day 4

In [207]:
input = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX""".split('\n')
input = get_input(4)

def day_four(a, two=False):
    count = 0        
    for y in range(len(a)):
        for x in range(len(a[0])):
            c = a[y][x]
            if not two and c == 'X':
                for y0 in (-1,0,1):
                    for x0 in (-1,0,1):
                        if y0 < 0 and y < 3: continue
                        if x0 < 0 and x < 3: continue
                        try:
                            if c + a[y+y0][x+x0] + a[y+y0*2][x+x0*2] + a[y+y0*3][x+x0*3] == 'XMAS':
                                count += 1
                        except IndexError:
                            pass
            elif two and c == 'A' and y > 0 and x > 0:
                try:
                    first = a[y-1][x-1] + a[y+1][x+1]
                    second = a[y-1][x+1] + a[y+1][x-1]
                    if 'M' in first and 'S' in first and 'M' in second and 'S' in second:
                        count += 1
                except IndexError:
                    pass
    return count

output(day_four(input), ts=1733290413)
output(day_four(input, True), ts=1733290716)

2336                                                              [00:33:33]
1831                                                              [00:38:36]


# Day 5

In [238]:
input = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47""".split('\n')
input = get_input(5)

def day_five(a, two=False):
    rules = []
    updates = []
    in_rules = True
    for row in a:
        if not row:
            in_rules = False
            continue
        
        if in_rules:
            rules.append(row)
        else:
            updates.append(row)
        
    updates = [i.split(',') for i in updates]
    middle = 0
    for update in updates:
        i = 0
        fail = False
        while i < len(update):
            page = update[i]
            j = 0
            while j < len(update):
                other = update[j]
                if j < i:
                    if f"{page}|{other}" in rules:
                        fail = True
                        if two:
                            update[i] = other
                            update[j] = page
                            i = j
                            page = other
                        else:
                            break
                elif j > i:
                    if f"{other}|{page}" in rules:
                        fail = True
                        if two:
                            update[i] = other
                            update[j] = page
                            j = i
                            page = other
                        else:
                            break
                j += 1
            else:
                i += 1
                continue
            break
        if two and fail:
            middle += int(update[len(update)//2])
        elif not two and not fail:
            middle += int(update[len(update)//2])
        
    return middle

output(day_five(input), ts=1733375557)
output(day_five(input, True), ts=1733376076)

5651                                                              [00:12:37]
4743                                                              [00:21:16]


# Day 6

In [105]:
input = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...""".split('\n')
input = get_input(6)

UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
rotate = {
    UP: RIGHT,
    RIGHT: DOWN,
    DOWN: LEFT,
    LEFT: UP
}

def make_grid(a):
    y0 = 0
    for row in a:
        if '^' in row:
            x0 = row.index('^')
            break
        y0 += 1
    
    grid = np.zeros((len(a),len(a[0])), np.int16)
    y = 0
    for row in a:
        x = 0
        for c in row:
            if c == '#':
                grid[y][x] = -1
            x += 1
        y += 1
    
    grid[y0][x0] += 1
    return grid, x0, y0
    
def run_path(grid, x0, y0):
    d = (-1, 0)
    visited = last_visited = counter = i = 0
    while i < 10000:
        i += 1
        y1 = y0 + d[0]
        x1 = x0 + d[1]
        if y1 < 0 or x1 < 0:
            break
        try:
            if grid[y1][x1] < 0:
                d = rotate[d]
                continue

            grid[y1][x1] += 1
            y0 = y1
            x0 = x1
        except IndexError:
            break
        
        #if i > 5000:
        #    visited = len(np.where(grid>0)[0])
        #    if visited == last_visited:
        #        counter += 1
        #        if counter > 50:
        #            return None
        #    else:
        #        last_visited = visited
        #        counter = 0
    else:
        return None
    
    return grid

def day_six_brute(a, two=False):
    grid, x0, y0 = make_grid(a)
    if two:
        cnt = 0
        grid = run_path(grid, x0, y0)
        ys, xs = np.where(grid>0)
        for i, (y, x) in enumerate(zip(ys, xs)):
            if i%100==0:print(i)
            if y == y0 and x == x0: continue
            grid, _, _ = make_grid(a)
            grid[y][x] = -1
            grid = run_path(grid, x0, y0)
            if grid is None:
                cnt += 1
        return cnt
        
    else:
        grid = run_path(grid, x0, y0)
        grid[np.where(grid>0)] = 1
        grid[np.where(grid<0)] = 0
        return np.sum(grid)

def make_graph(a):
    y0 = 0
    for row in a:
        if '^' in row:
            x0 = row.index('^')
            break
        y0 += 1
    
    graph = nx.DiGraph()
    m = len(a)
    n = len(a[0])
    for y in range(m):
        for x in range(n):
            if a[y][x] == '#':
                continue
            elif a[y][x] == '^':
                start = (y, x, UP)
            
            for d in (UP, DOWN, LEFT, RIGHT):
                node = (y, x, d)
                y1 = y + d[0]
                x1 = x + d[1]
                
                if y1 < 0 or y1 >= m or x1 < 0 or x1 >= n:
                    continue
                
                if a[y1][x1] == '#':
                    graph.add_edge(node, (y, x, rotate[d]))
                else:
                    graph.add_edge(node, (y1, x1, d))
    return graph, start

def day_six(a, two=False):
    graph, start = make_graph(a)
    if two:
        return
    node = [start]
    visited = set()
    while node:
        node = node[0]
        visited.add((node[0],node[1]))
        node = graph.neighbors(node)
        node = list(node)
    return len(visited)

%timeit output(day_six(input), ts=1638334897)
%timeit output(day_six_brute(input))
#output(day_six(input, True), ts=163335082) #2279

5534                                                              [00:01:37]
5534                                                              [00:01:37]
5534                                                              [00:01:37]
5534                                                              [00:01:37]
5534                                                              [00:01:37]
5534                                                              [00:01:37]
5534                                                              [00:01:37]
5534                                                              [00:01:37]
401 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5534
5

# Day 7

In [75]:
input = """190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20""".split('\n')
input = get_input(7)

def can_solve(test, vals, two):
    if not vals:
        return False
    
    arg = vals[-1]
    if test == arg:
        return True
    
    if test % arg == 0:
        if can_solve(test // arg, vals[:-1], two):
            return True
    
    if test > arg:
        if can_solve(test - arg, vals[:-1], two):
            return True
    
    if two and str(test).endswith(str(arg)):
        if can_solve(int(str(test)[:-len(str(arg))]), vals[:-1], two):
            return True
    
    return False

def day_seven(a, two=False):
    total = 0
    for row in a:
        test, vals = row.split(':')
        test = int(test)
        vals = [int(i) for i in vals.strip().split()]
        if can_solve(test, vals, two):
            total += test
    return total

def day_seven_rubbish(a, two=False):
    total = 0
    for idx, row in enumerate(a):
        print(idx,row)
        test, vals = row.split(':')
        test = int(test)
        vals = vals.strip().split()
        #print(test,vals)
        for combo in more_itertools.distinct_permutations('+' * len(vals) + '*' * len(vals), len(vals)-1):
            val = vals[0]
            for op, arg in zip(combo, vals[1:]):
                val = eval(f'{val}{op}{arg}')
            #print(val)
            if val == test:
                #print('success')
                total += int(test)
                break
        else:
            if two:
                for combo in more_itertools.distinct_permutations('+' * len(vals) + '*' * len(vals) + '|' * len(vals), len(vals)-1):
                    if '|' not in combo: continue
                    val = vals[0]
                    for op, arg in zip(combo, vals[1:]):
                        if op == '|':
                            expr = f'{val}{arg}'
                        else:
                            expr = f'{val}{op}{arg}'
                        val = eval(expr)
                        i += 1
                    #print(val)
                    if val == test:
                        #print('success')
                        total += int(test)
                        break
    return total
        

output(day_seven(input), ts=1638334897)
output(day_seven(input, True), ts=1638335082)

3119088655389                                                     [00:01:37]
264184041398847                                                   [00:04:42]


# Day 8

In [123]:
input = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............""".split('\n')
input = get_input(8)

def in_grid(node, m, n):
    x, y = node
    return x >= 0 and x < n and y >= 0 and y < m

def day_eight(a, two=False):
    ants = {}
    m = len(a)
    n = len(a[0])
    for y in range(m):
        for x in range(n):
            c = a[y][x]
            if c == '.': continue
            if c in ants:
                ants[c].append((x,y))
            else:
                ants[c] = [(x,y)]
    #print(ants)
    
    antinodes = set()
    for freq in ants:
        nodes = ants[freq]
        num = len(nodes)
        if num < 2: continue
        for i in range(num):
            for j in range(i+1, num):
                x0, y0 = nodes[i]
                x1, y1 = nodes[j]
                dx = x0 - x1
                dy = y0 - y1
                
                for node in ((x0+dx, y0+dy), (x1-dx, y1-dy)):
                    if in_grid(node,m,n):
                        antinodes.add(node)
                
                if two:
                    x, y = nodes[i]
                    while in_grid((x,y),m,n):
                        antinodes.add((x,y))
                        x += dx
                        y += dy
                        
                    x, y = nodes[j]
                    while in_grid((x,y),m,n):
                        antinodes.add((x,y))
                        x -= dx
                        y -= dy
                        
    return len(antinodes)

output(day_eight(input), ts=1638334897)
output(day_eight(input, True), ts=1638335082)

289                                                               [00:01:37]
1030                                                              [00:04:42]


# Day 9

In [212]:
test = """2333133121414131402""".split('\n')
input = get_input(9)

def day_nine(a, two=False):
    row = a[0]
    first = 0
    last = -1
    total = 0
    space = 0
    sum_idx = 0
    file_id = 0
    last_file_id = len(row) // 2
    disk=''
    remaining = 0
    used_ids = set()
    while True:
        if two and file_id in used_ids:
            c = int(row[first])
            disk += '.' * c
            sum_idx += c
        else:
            used_ids.add(file_id)
            c = int(row[first])
            for i in range(c):
                #disk += str(file_id)
                total += sum_idx * file_id
                sum_idx += 1
        file_id += 1
        first += 1

        space = int(row[first])
        first += 1

        if two:
            last = -1
            last_file_id = len(row) // 2
        while space > 0:
            if last_file_id in used_ids:
                last -= 2
                last_file_id -= 1
                continue

            if -last >= len(row):
                last = -1
                sum_idx += space
                #disk += '.' * space
                break

            c = remaining if remaining else int(row[last])
            if c <= space or not two:
                for i in range(c):
                    space -= 1
                    if space < 0:
                        space = 0
                        remaining = c-i
                        break
                    #disk += str(last_file_id)
                    total += sum_idx * last_file_id
                    sum_idx += 1
                else:
                    used_ids.add(last_file_id)
                    last_file_id -= 1
                    last -= 2
                    remaining = 0
            else:
                last_file_id -= 1
                last -= 2
                    

            if two and first - last >= len(row):
                last = -1
                sum_idx += space
                #disk += '.' * space
                break
        
        if first - last >= len(row):
            break
    
    while remaining:
        #disk += str(last_file_id)
        total += sum_idx * last_file_id
        sum_idx += 1
        remaining -= 1
    
    #print(disk) # 0099811188827773336446555566
    return total       

output(day_nine(test), ts=1638334897)
output(day_nine(test, True), ts=1638335082)
output(day_nine(input), ts=1638334897)
output(day_nine(input, True), ts=1638335082)

1928                                                              [00:01:37]
2858                                                              [00:04:42]
6386640365805                                                     [00:01:37]
6423258376982                                                     [00:04:42]


# Day 10

In [261]:
input="""89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732""".split('\n')
input = get_input(10)
    
def make_graph(a):
    graph = nx.DiGraph()
    m = len(a)
    n = len(a[0])
    for y in range(m):
        for x in range(n):
            node = (y,x)
            val = int(a[y][x])
            graph.add_node(node, val=val)
            for d in (UP, DOWN, LEFT, RIGHT):
                y1 = y + d[0]
                x1 = x + d[1]
                
                if y1 < 0 or y1 >= m or x1 < 0 or x1 >= n:
                    continue
                
                if int(a[y1][x1]) == val + 1:
                    graph.add_node((y1,x1), val=val+1)
                    graph.add_edge(node, (y1,x1))
    return graph


def day_ten(a, two=False):
    g = make_graph(a)
    total = 0
    heads = [n for n,v in g.nodes(data='val') if v == 0]
    tails = [n for n,v in g.nodes(data='val') if v == 9]
    for h in heads:
        d = nx.descendants(g, h)
        for t in tails:
            if t in d:
                if two:
                    total += sum(1 for _ in nx.all_simple_paths(g, h, t))
                else:
                    total += 1
    return total

output(day_ten(input), ts=1638334897)
output(day_ten(input, True), ts=1638335082)

646                                                               [00:01:37]
1494                                                              [00:04:42]


# Day 11

In [44]:
test = """125 17""".split('\n')
input = get_input(11)

def blink(s,d):
    if s in d:
        return d[s]
    if s == 0:
        out = 1
    elif len(str(s)) % 2 == 0:
        s0 = str(s)
        l = len(s0)
        out = (int(s0[:l//2]),int(s0[l//2:]))
    else:
        out = s * 2024
    d[s] = out
    return out

def day_eleven(a, two=False):
    stones = {int(i):1 for i in a[0].split()}
    memory = {}
    if two:
        num = 75
    else:
        num = 25
    for i in range(num):
        update = {}
        for s in stones:
            out = blink(s, memory)
            if type(out) == tuple:
                for o in out:
                    update[o] = update.get(o,0) + stones[s]
            else:
                update[out] = update.get(out,0) + stones[s]
        stones = update
    return sum(i for i in stones.values())
    
output(day_eleven(test), ts=1638334897)
output(day_eleven(test, True), ts=1638335082)
output(day_eleven(input), ts=1638334897)
output(day_eleven(input, True), ts=1638335082)

55312                                                             [00:01:37]
65601038650482                                                    [00:04:42]
193269                                                            [00:01:37]
228449040027793                                                   [00:04:42]


# Day 12

In [219]:
test1 = """AAAA
BBCD
BBCC
EEEC""".split('\n')
test2 = """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO""".split('\n')
test3 = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE""".split('\n')
test4 = """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE""".split('\n')
test5 = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA""".split('\n')
input = get_input(12)

UP = (-1,0)
DOWN = (1,0)
LEFT = (0,-1)
RIGHT = (0,1)
rotate = {
    UP: RIGHT,
    RIGHT: DOWN,
    DOWN: LEFT,
    LEFT: UP
}
antirotate = {
    UP: LEFT,
    LEFT: DOWN,
    DOWN: RIGHT,
    RIGHT: UP
}

def in_grid(x,y,m,n):
    return x >= 0 and y >= 0 and x < n and y < m

def get_perim(i):
    m, n = i.shape
    p = 0
    y = x = 0
    while not i[y][x]:
        x += 1
    x0 = x
    y0 = y
    p += 1
    d = RIGHT
    b = UP
    #print(y0,x0,RIGHT)
    while True:
        if x == x0 and y == y0 and d == UP:
            break

        y1 = y + d[0]
        x1 = x + d[1]
        if not in_grid(x1,y1,m,n):
            #print('off grid',y,x,d)
            d = rotate[d]
            b = rotate[b]
            p += 1
            continue

        y2 = y1 + b[0]
        x2 = x1 + b[1]
        #print(y,x,y2,x2)
        if in_grid(x2,y2,m,n) and i[y2][x2]:
            #print('inside corner',y,x,d)
            d = antirotate[d]
            b = antirotate[b]
            p += 1
            x = x2
            y = y2
            continue
        
        if not i[y1][x1]:
            #print('false',y,x,d)
            d = rotate[d]
            b = rotate[b]
            p += 1
            continue

        x = x1
        y = y1
    return p

def day_twelve(a, two=False):
    m = len(a)
    n = len(a[0])
    grid = np.zeros((m,n), np.uint8)
    for y in range(m):
        for x in range(n):
            grid[y][x] = ord(a[y][x])
    img = sk_measure.label(grid, connectivity=1)
    
    total = 0
    for j,r in enumerate(sk_measure.regionprops(img)):
        i = r.image
        m,n = i.shape
        p = 0
        if two:
            p = get_perim(i)
            
            q = np.ones((m,n))
            q[np.where(i)] += 1
            img2 = sk_measure.label(q,connectivity=1)
            for idx, r2 in enumerate(sk_measure.regionprops(img2)):
                for y,x in r2.coords:
                    if y == 0 or y == m-1 or x == 0 or x == n-1:
                        break
                else:
                    p += get_perim(r2.image)
            total += p * r.num_pixels
        else:
            for y in range(m):
                for x in range(n):
                    if i[y][x]:
                        for y1,x1 in ((y-1,x),(y+1,x),(y,x-1),(y,x+1)):
                            if x1 < 0 or x1 >= n or y1 < 0 or y1 >= m:
                                p += 1
                            elif not i[y1][x1]:
                                p += 1
            if p == 0:
                p = 4
            total += p * r.num_pixels
    return total
            
#output(day_twelve(test1), ts=1638334897)
#output(day_twelve(test2), ts=1638334897)
#output(day_twelve(test3), ts=1638334897)
output(day_twelve(input), ts=1638334897)
#output(day_twelve(test1, True), ts=1638335082)
#output(day_twelve(test2, True), ts=1638335082)
#output(day_twelve(test4, True), ts=1638335082)
#output(day_twelve(test5, True), ts=1638335082)
#output(day_twelve(test3, True), ts=1638335082)
output(day_twelve(input, True), ts=1638335082)

1473276                                                           [00:01:37]
901100                                                            [00:04:42]


# Day 13

In [255]:
test = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279""".split('\n')
input = get_input(13)


def find_prize(a, b, x0, y0, two=False):
    x = y = na = nb = 0
    if two:
        x0 += 10000000000000
        y0 += 10000000000000
    adx, ady = a
    bdx, bdy = b
    while x < x0 and y < y0:
        dx = x0-x
        dy = y0-y
        if dx % bdx == 0 and dy % bdy == 0 and dx // bdx == dy // bdy:
            nb = dx // bdx
            tokens = na * 3 + nb
            break
        x += adx
        y += ady
        na += 1
    else:
        return 0
    
    return tokens
   
"""
na * ax + nb * bx = x0
nb = (x0 - na*ax) / bx

na * ay + nb * by = y0
nb = (y0 - na*ay) / by

x0 - na*ax / bx = y0 - na*ay / by
x0*by - na*ax*by = y0*bx - na*ay*bx
x0*by - y0*bx = na*ax*by - na*ay*bx
na = (x0*by - y0*bx) / (ax*by - ay*bx)
"""
def find_prize(a, b, x0, y0, two=False):
    x = y = na = nb = 0
    if two:
        x0 += 10000000000000
        y0 += 10000000000000
    ax, ay = a
    bx, by = b
    
    na = (x0*by - y0*bx) / (ax*by - ay*bx)
    nb = (x0 - na*ax) / bx
    #print(na,nb)
    if int(na) != na or int(nb) != nb:
        return 0
    return int(na * 3 + nb)

    
    
def day_thirteen(a, two=False):
    total = 0
    for row in a:
        if 'Prize' in row:
            _, x, y = row.split()
            x = int(x.split('=')[1][:-1])
            y = int(y.split('=')[1])
            total += find_prize(A, B, x, y, two)
        elif row:
            _, _, x, y = row.split()
            x = int(x.split('+')[1][:-1])
            y = int(y.split('+')[1])
            if 'A' in row:
                A = (x,y)
            else:
                B = (x,y)
        else:
            continue
    return total
            
        

output(day_thirteen(test), ts=1638334897)
output(day_thirteen(test, True), ts=1638335082)
output(day_thirteen(input), ts=1638334897)
output(day_thirteen(input, True), ts=1638335082)

480                                                               [00:01:37]
875318608908                                                      [00:04:42]
37680                                                             [00:01:37]
87550094242995                                                    [00:04:42]


# Day 14

In [390]:
test = """p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3""".split('\n')
input = get_input(14)

def tick(robots, m, n):
    update = []
    for x, y, dx, dy in robots:
        update.append(((x+dx) % n, (y+dy) % m, dx, dy))
    return update

def get_quads(robots, m, n):
    x0 = n // 2
    y0 = m // 2
    a = b = c = d = 0
    for x, y, _, _ in robots:
        if x == x0 or y == y0:
            continue
        if x < x0:
            if y < y0:
                a += 1
            else:
                b += 1
        else:
            if y < y0:
                c += 1
            else:
                d += 1
    return a, b, c, d

def day_fourteen(a, m, n, two=False):
    robots = []
    for row in a:
        p, v = row.split()
        x, y = [int(i) for i in p[2:].split(',')]
        dx, dy = [int(i) for i in v[2:].split(',')]
        robots.append((x, y, dx, dy))
    
    if two:
        num = len(robots)
        for i in range(10000):
            robots = tick(robots, m, n)
            if max(get_quads(robots, m, n)) > num // 2:
                return i+1
                ar = np.zeros((m,n),int)
                for x,y,_,_ in robots:
                    ar[y][x] += 1
                print(i)
                for y in range(m):
                    row = ''
                    for x in range(n):
                        row += '.' if ar[y][x] == 0 else str(ar[y][x])
                    print(row)
        return "FAIL"
            
    
    for i in range(100):
        robots = tick(robots, m, n)
    return math.prod(get_quads(robots, m, n))
        

output(day_fourteen(test, 7, 11), ts=1638334897)
output(day_fourteen(input, 103, 101), ts=1638334897)
output(day_fourteen(input, 103, 101, True), ts=1638335082)

12                                                                [00:01:37]
214400550                                                         [00:01:37]
8149                                                              [00:04:42]


# Day 15

In [346]:
test = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<""".split('\n')
test2 = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^""".split('\n')
test3 = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^""".split('\n')
input = get_input(15)

MOVES = {
    '^': (-1,0),
    '>': (0,1),
    'v': (1,0),
    '<': (0,-1)
}

def print_grid(grid):
    m, n = grid.shape
    for y in range(m):
        row = ''
        for x in range(n):
            if grid[y][x] == 9:
                row += '@'
            elif grid[y][x] == -1:
                row += '#'
            elif grid[y][x] == 1:
                row += 'O'
            else:
                row += '.'
        print(row)
    print()
            

def make_move(x, y, d, grid):
    dy, dx = MOVES[d]
    x1 = x + dx
    y1 = y + dy
    c = grid[y1][x1]
    
    if c == -1:
        return x, y
    elif c == 0:
        grid[y][x] = 0
        grid[y1][x1] = 9
        return x1, y1
    else:
        while c == 1:
            x1 += dx
            y1 += dy
            c = grid[y1][x1]
        if c == 0:
            while x1 != x or y1 != y:
                grid[y1][x1] = 1
                x1 -= dx
                y1 -= dy
            x1 = x + dx
            y1 = y + dy
            grid[y][x] = 0
            grid[y1][x1] = 9
            return x1, y1
        else:
            return x, y

def day_fifteen(a, two=False):
    grid_in = []
    moves = []
    grid = None
    for row in a:
        if not row:
            m = len(grid_in)
            n = len(grid_in[0])
            grid = np.zeros((m,n), np.int8)
            for y in range(m):
                for x in range(n):
                    if grid_in[y][x] == '#':
                        grid[y][x] = -1
                    elif grid_in[y][x] == 'O':
                        grid[y][x] = 1
                    elif grid_in[y][x] == '@':
                        grid[y][x] = 9
                        x0 = x
                        y0 = y
            continue
        if grid is None:
            grid_in.append(row)
        else:
            moves.append(row)
    
    x = x0
    y = y0
    #print_grid(grid)
    for row in moves:
        for move in row:
            x, y = make_move(x, y, move, grid)
            #print_grid(grid)
    #print_grid(grid)
    
    total = 0
    ys, xs = np.where(grid==1)
    for y, x in zip(ys, xs):
        total += 100 * y + x
    return total

def print_grid2(grid):
    m, n = grid.shape
    for y in range(m):
        row = ''
        for x in range(n):
            if grid[y][x] == -2:
                row += '@'
            elif grid[y][x] == -1:
                row += '#'
            elif grid[y][x] == 1:
                row += '['
            elif grid[y][x] == 2:
                row += ']'
            else:
                row += '.'
        print(row)
    print()
            

def make_move2(x, y, d, grid):
    dy, dx = MOVES[d]
    x1 = x + dx
    y1 = y + dy
    c = grid[y1][x1]
    
    if c == -1:
        return x, y, False
    elif c == 0:
        grid[y][x] = 0
        grid[y1][x1] = -2
        return x1, y1, True
    elif dy == 0:
        while c > 0:
            x1 += dx
            y1 += dy
            c = grid[y1][x1]
        if c == 0:
            v = 1 if dx < 0 else 2
            while x1 != x:
                grid[y1][x1] = v
                v = 2 if v == 1 else 1
                x1 -= dx
            x1 = x + dx
            grid[y][x] = 0
            grid[y1][x1] = -2
            return x1, y1, True
        else:
            return x, y, False
    else:
        boxes = set()
        if c == 1:
            boxes.add((y1,x))
        else:
            boxes.add((y1,x-1))
        while True:
            #print(boxes)
            y2 = y1 + dy
            clear = True
            for by, bx in boxes:
                if by != y1:
                    continue
                c1 = grid[y2][bx]
                c2 = grid[y2][bx+1]
                if c1 < 0 or c2 < 0:
                    return x, y, False
                if c1 > 0 or c2 > 0:
                    # do not break here because another box may be blocked entirely
                    clear = False
                    
            if clear:
                reverse = dy > 0
                for y1, x1 in sorted(list(boxes), key=lambda c:c[0], reverse=reverse):
                    grid[y1][x1] = 0
                    grid[y1][x1+1] = 0
                    y2 = y1 + dy
                    grid[y2][x1] = 1
                    grid[y2][x1+1] = 2
                y1 = y + dy
                grid[y][x] = 0
                grid[y1][x] = -2
                return x, y1, True
            
            for by, bx in list(boxes):
                if by != y1:
                    continue
                if grid[y2][bx] == 1:
                    boxes.add((y2,bx))
                elif grid[y2][bx] == 2:
                    boxes.add((y2,bx-1))
                if grid[y2][bx+1] == 1:
                    boxes.add((y2,bx+1))
            
            y1 = y2
            

def day_fifteen2(a, two=False):
    grid_in = []
    moves = []
    grid = None
    for row in a:
        if not row:
            m = len(grid_in)
            n = len(grid_in[0] * 2)
            grid = np.zeros((m,n), np.int8)
            for y in range(m):
                for x in range(n // 2):
                    if grid_in[y][x] == '#':
                        grid[y][2*x] = -1
                        grid[y][2*x + 1] = -1
                    elif grid_in[y][x] == 'O':
                        grid[y][2*x] = 1
                        grid[y][2*x + 1] = 2
                    elif grid_in[y][x] == '@':
                        grid[y][2*x] = -2
                        grid[y][2*x + 1] = 0
                        x0 = 2*x
                        y0 = y
            continue
        if grid is None:
            grid_in.append(row)
        else:
            moves.append(row)
    
    x = x0
    y = y0
    #print_grid2(grid)
    for i, row in enumerate(moves):
        for j, move in enumerate(row):
            x, y, moved = make_move2(x, y, move, grid)
            #print(i * len(row) + j, move, moved)
            #print_grid2(grid)
    #print_grid2(grid)
    
    total = 0
    ys, xs = np.where(grid==1)
    for y, x in zip(ys, xs):
        total += 100 * y + x
    return total
        
        
    

output(day_fifteen(test), ts=1638334897)
output(day_fifteen(test2), ts=1638334897)
output(day_fifteen(input), ts=1638334897)
output(day_fifteen2(test3, True), ts=1638335082)
output(day_fifteen2(test2, True), ts=1638335082)
output(day_fifteen2(input, True), ts=1638335082)

2028                                                              [00:01:37]
10092                                                             [00:01:37]
1436690                                                           [00:01:37]
618                                                               [00:04:42]
9021                                                              [00:04:42]
1482350                                                           [00:04:42]


# Day 16

In [468]:
test = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############""".split('\n')
test2 = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################""".split('\n')
input = get_input(16)

UP = (-1,0)
DOWN = (1,0)
LEFT = (0,-1)
RIGHT = (0,1)
clockwise = {
    UP: RIGHT,
    RIGHT: DOWN,
    DOWN: LEFT,
    LEFT: UP
}
anticlockwise = {
    UP: LEFT,
    LEFT: DOWN,
    DOWN: RIGHT,
    RIGHT: UP
}

def get_pts(pt, grid):
    x, y, d = pt
    right = clockwise[d]
    left = anticlockwise[d]
    x1 = x + d[1]
    y1 = y + d[0]
    out = [[(x, y, right), 1000], [(x, y, left), 1000]]
    if grid[y1][x1] != '#':
        out.append([(x1,y1,d), 1])
    return out

def day_sixteen(a, two=False):
    m = len(a)
    n = len(a[0])
    limit = m*n*1000
    x0 = 1
    y0 = m - 2
    X = n - 2
    Y = 1
    E1 = (X,Y,UP)
    E2 = (X,Y,RIGHT)
    
    bfs = {(x0,y0,RIGHT): 0}
    paths = [[(x0,y0,RIGHT)]]
    done = False
    while not done:
        done = True
        new_paths = []
        for i, path in enumerate(list(paths)):
            pt = path[-1]
            if pt == E1 or pt == E2:
                new_paths.append(path)
                continue
            x, y, d = pt
            score = bfs[pt]
            for p, inc in get_pts(pt, a):
                if score + inc <= bfs.get(p, limit):
                    bfs[p] = score + inc
                    new_paths.append(path + [p])
                    done = False
        paths = new_paths
    
    best = min(bfs.get(E1, limit), bfs.get(E2, limit))
    if best == limit: raise
    
    if bfs.get(E1, limit) == best:
        end = E1
    else:
        end = E2
        
    tiles = set()
    for path in paths:
        if path[-1] != end:
            continue
        path_tiles = []
        score = -1000
        last = path[0]
        for pt in path:
            if pt[0] == last[0] and pt[1] == last[1]:
                score += 1000
            else:
                score += 1
            last = pt
            
            x, y, d = pt
            path_tiles.append((x,y))
        
        if score == best:
            tiles = tiles.union(path_tiles)
            
    
    return best, len(tiles)

def day_sixteen2(a, two=False):
    m = len(a)
    n = len(a[0])
    g = nx.DiGraph()
    x0 = 1
    y0 = m - 2
    S = (x0, y0, RIGHT)
    X = n - 2
    Y = 1
    E1 = (X,Y,UP)
    E2 = (X,Y,RIGHT)
    
    for y in range(1, m-1):
        for x in range(1, n-1):
            if a[y][x] == '#':
                continue
            for d in clockwise:
                g.add_edge((x,y,d), (x,y,clockwise[d]), weight=1000)
                g.add_edge((x,y,d), (x,y,anticlockwise[d]), weight=1000)
                dy, dx = d
                x1 = x + dx
                y1 = y + dy
                if a[y1][x1] != '#':
                    g.add_edge((x,y,d), (x1,y1,d), weight=1)

    cost1 = nx.shortest_path_length(g, S, E1, 'weight')
    cost2 = nx.shortest_path_length(g, S, E2, 'weight')
    if cost1 < cost2:
        end = E1
        cost = cost1
    else:
        end = E2
        cost = cost2
    
    if two:
        tiles = set()
        for path in nx.all_shortest_paths(g, S, end, 'weight'):
            for pt in path:
                x, y, d = pt
                tiles.add((x,y))
        return len(tiles)
    else:
        return cost
                    

output(day_sixteen2(test), ts=1638334897)
output(day_sixteen2(test2), ts=1638334897)
output(day_sixteen2(input), ts=1638334897)
output(day_sixteen2(test, True), ts=1638334897)
output(day_sixteen2(test2, True), ts=1638334897)
output(day_sixteen2(input, True), ts=1638334897)

7036                                                              [00:01:37]
11048                                                             [00:01:37]
79404                                                             [00:01:37]
45                                                                [00:01:37]
64                                                                [00:01:37]
451                                                               [00:01:37]


# Day 17

In [608]:
test = """Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0""".split('\n')
test2 = """Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0""".split('\n')
input = get_input(17)

out = []

def combo(op, reg):
    if op < 4:
        return op
    elif op == 4:
        return reg['A']
    elif op == 5:
        return reg['B']
    elif op == 6:
        return reg['C']
    else:
        raise

#Program: 2,4,1,3,7,5,0,3,4,3,1,5,5,5,3,0
#2,4 b = a % 8
#1,3 b = b ^ 3
#7,5 c = a // 2**b
#0,3 a = a // 8
#4,3 b = b ^ c
#1,5 b = b ^ 5
#5,5 out = b % 8
#3,0 jump
def run_inst(inst, op, reg):
    if inst == 0:
        reg['A'] = reg['A'] // 2**combo(op, reg)
    elif inst == 1:
        reg['B'] = reg['B'] ^ op
    elif inst == 2:
        reg['B'] = combo(op, reg) % 8
    elif inst == 3:
        if reg['A'] != 0:
            return op
    elif inst == 4:
        reg['B'] = reg['B'] ^ reg['C']
    elif inst == 5:
        out.append(combo(op, reg) % 8)
    elif inst == 6:
        reg['B'] = reg['A'] // 2**combo(op, reg)
    elif inst == 7:
        reg['C'] = reg['A'] // 2**combo(op, reg)

def run_prog(prog, reg, two):       
    idx = 0
    while idx < len(prog):
        inst = prog[idx]
        op = prog[idx+1]
        ret = run_inst(inst, op, reg)
        if ret is not None:
            idx = ret
        else:
            idx += 2
        if two and inst == 5:
            return out[0]

def run_recur(prog, start=0, end=8, inst=None, a=0):
    global out
    if inst is None:
        inst = len(prog)-1
    if inst < 0:
        return a
    
    val = prog[inst]
    reg = {}
    for j in range(start, end):
        reg['A'] = j
        reg['B'] = 0
        reg['C'] = 0
        out = []
        if run_prog(prog, reg, True) == val:
            reg['A'] = j
            reg['B'] = 0
            reg['C'] = 0
            out = []
            run_prog(prog, reg, False)
            if out != prog[-len(out):]:
                continue
            a = run_recur(prog, j*8, (j+7)*8, inst-1, j)
            if a != 0:
                return a
    return 0

def day_seventeen(a, two=False):
    global out
    out = []
    reg = {}
    prog = []
    for row in a:
        if not row:
            continue
        if 'Register' in row:
            _, r, n = row.split()
            r = r[0]
            n = int(n)
            reg[r] = n
        elif 'Program' in row:
            _, p = row.split()
            prog = [int(i) for i in p.split(',')]

    if two:
        return run_recur(prog)
    else:
        run_prog(prog, reg, two)
        return ','.join([str(i) for i in out])
        
output(day_seventeen(test))
output(day_seventeen(input))
output(day_seventeen(test2, True))
output(day_seventeen(input, True))

4,6,3,5,6,3,5,2,1,0
2,0,1,3,4,0,2,1,7
117440
236580836040301


# Day 18

In [631]:
test = """5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0""".split('\n')
input = get_input(18)

def day_eighteen(a, sz, two=False):
    g = nx.Graph()
    grid = np.zeros((sz, sz), np.uint8)
    for y in range(sz):
        for x in range(sz):
            for d in (UP, DOWN, LEFT, RIGHT):
                x1 = x + d[1]
                y1 = y + d[0]
                if x1 < 0 or x1 == sz or y1 < 0 or y1 == sz:
                    continue
                g.add_edge((x,y), (x1,y1))
    num = 12 if sz < 70 else 1024
    if two:
        for i, row in enumerate(a):
            x, y = [int(c) for c in row.split(',')]
            g.remove_node((x,y))
            if i < num:
                continue
            try:
                nx.shortest_path_length(g, (0,0), (sz-1,sz-1))
            except:
                return f'{x},{y}'
    else:
        for i in range(num):
            x, y = [int(c) for c in a[i].split(',')]
            g.remove_node((x,y))
        return nx.shortest_path_length(g, (0,0), (sz-1,sz-1))

output(day_eighteen(test, 7), ts=1638334897)
output(day_eighteen(input, 71), ts=1638334897)
output(day_eighteen(test, 7, True), ts=1638335082)
output(day_eighteen(input, 71, True), ts=1638335082)

22                                                                [00:01:37]
404                                                               [00:01:37]
6,1                                                               [00:04:42]
27,60                                                             [00:04:42]


# Day 19

In [353]:
test = """r, wr, b, g, bwu, rb, gb, br

brwrr
bggr
gbbr
rrbgbr
ubwu
bwurrg
brgr
bbrgwb""".split('\n')
input = get_input(19)

colors = {
    'w': '1',
    'u': '2',
    'b': '3',
    'r': '4',
    'g': '5'
}
def find_pattern(num, patterns, limit, count=0):
    for p in patterns:
        if num == p:
            return True
        if num < p:
            return False
        for n in range(1, limit+1):
            if num % 10**n == p:
                if find_pattern(num // 10**n, patterns, limit):
                    return True
    return False

memory = {}
def find_perms(num, patterns, limit):
    count = 0
    if num in memory:
        return memory[num]
    
    for p in patterns:
        if num == p:
            count += 1
        elif num < p:
            break
        for n in range(1, limit+1):
            if num % 10**n == p and num != p:
                count += find_perms(num // 10**n, patterns, limit)
    
    memory[num] = count
    return count

def day_nineteen(a, two=False):
    patterns = [s.strip() for s in a[0].split(',')]
    for c in colors:
        update = []
        for p in patterns:
            update.append(p.replace(c, colors[c]))
        patterns = update
    count = 0
    total = 0
    for row in a[2:]:
        for c in colors:
            row = row.replace(c, colors[c])
        row_pats = []
        limit = 0
        for p in patterns:
            if p in row:
                row_pats.append(int(p))
                limit = max(limit, len(p))
        perms = find_perms(int(row), sorted(row_pats), limit)
        if perms > 0:
            count += 1
        total += perms
    return count, total

#output(day_nineteen(test), ts=1638334897)
#output(day_nineteen(input), ts=1638334897)
output(day_nineteen(test, True), ts=1638335082)
memory = {}
output(day_nineteen(input, True), ts=1638335082)    

(6, 16)                                                           [00:04:42]
(369, 761826581538190)                                            [00:04:42]


# Day 20

In [166]:
test = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############""".split('\n')
input = get_input(20)

UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)

def day_twenty(a, two=False):
    m = len(a)
    n = len(a[0])
    g = nx.Graph()
    for y in range(1,m-1):
        for x in range(1,n-1):
            if a[y][x] == 'S':
                start = (x,y)
            elif a[y][x] == 'E':
                end = (x,y)
            elif a[y][x] == '#':
                continue
            for d in (UP, DOWN, LEFT, RIGHT):
                y1 = y + d[0]
                x1 = x + d[1]
                if x1 > 0 and x1 < n and y1 > 0 and y1 < m:
                    if a[y1][x1] != '#':
                        g.add_edge((x,y), (x1,y1))
    dist = nx.shortest_path_length(g, start, end)
    return dist
    
    cheats = {}
    for y in range(1,m-1):
        for x in range(1,n-1):
            if a[y][x] != '#':
                continue
            nodes = []
            for d in (UP, DOWN, LEFT, RIGHT):
                y1 = y + d[0]
                x1 = x + d[1]
                if a[y1][x1] != '#':
                    g.add_edge((x,y), (x1,y1))
            if (x,y) in g:
                cheat = nx.shortest_path_length(g, start, end)
                if cheat < dist:
                    k = dist - cheat
                    cheats[k] = cheats.get(k, 0) + 1
                g.remove_node((x,y))
    
    print(cheats)
    count = 0
    for k in cheats:
        if k >= 100:
            count += cheats[k]
    return count
     
def day_twenty(a, two=False):
    M = len(a)
    N = len(a[0])
    grid = np.zeros((M,N), np.uint8)
    grid[0,:] = 2
    grid[:,0] = 2
    grid[M-1,:] = 2
    grid[:,N-1] = 2
    for y in range(1,M-1):
        for x in range(1,N-1):
            if a[y][x] == 'S':
                start = (x,y)
            elif a[y][x] == 'E':
                end = (x,y)
            elif a[y][x] == '#':
                grid[y][x] = 1

    path = {start: 0}
    node = start
    while True:
        if node == end:
            break
        x, y = node
        for x1, y1 in get_adjacent(x, y):
            if grid[y1][x1] == 0 and (x1,y1) not in path:
                path[(x1,y1)] = path[node] + 1
                node = (x1,y1)
    
    if not two:
        cheats = {}
        rows, cols = np.where(grid==1)
        for y, x in zip(rows, cols):
            for d in (UP, RIGHT):
                x1 = x + d[1]
                y1 = y + d[0]
                x2 = x - d[1]
                y2 = y - d[0]
                if (x1,y1) in path and (x2,y2) in path:
                    cheat = abs(path[(x1,y1)] - path[(x2,y2)]) - 2
                    cheats[cheat] = cheats.get(cheat, 0) + 1
    else:
        shortcuts = set()
        for node in path:
            x, y = node
            for y0 in range(21):
                y1 = y + y0
                y2 = y - y0
                for x0 in range(0, 21-y0):
                    x1 = x + x0
                    x2 = x - x0
                    for n in ((x1,y1), (x1,y2), (x2,y1), (x2,y2)):
                        if n in path:
                            shortcuts.add((node, n))

        cheats = {}
        for cut in shortcuts:
            n1 = cut[0]
            n2 = cut[1]
            x1, y1 = n1
            x2, y2 = n2
            dist = abs(x2-x1) + abs(y2-y1)
            save = path[n2] - path[n1] - dist
            cheats[save] = cheats.get(save, 0) + 1    
    
    max_save = max(cheats)
    if max_save < 100:
        if two:       
            out = []
            for i in range(50, 77, 2):
                out.append(cheats[i])
            assert out == [32, 31, 29, 39, 25, 23, 20, 19, 12, 14, 12, 22, 4, 3]
        else:
            out = []
            for k in sorted(cheats):
                out.append(cheats[k])
            assert out == [14, 14, 2, 4, 2, 3, 1, 1, 1, 1, 1]

    count = 0
    for k in sorted(cheats):
        if k >= 100:
            count += cheats[k]
    return count

output(day_twenty(test), ts=1638334897)
output(day_twenty(input), ts=1638334897)
output(day_twenty(test, True), ts=1638335082)
output(day_twenty(input, True), ts=1638335082)

0                                                                 [00:01:37]
1441                                                              [00:01:37]
0                                                                 [00:04:42]
1021490                                                           [00:04:42]


# Day 21

In [749]:
"""
+---+---+---+
| 7 | 8 | 9 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
    | 0 | A |
    +---+---+

    +---+---+
    | ^ | A |
+---+---+---+
| < | v | > |
+---+---+---+
029A: <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
980A: <v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A
179A: <v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
456A: <v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A
379A: <v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A

"""
test = """029A
980A
179A
456A
379A""".split('\n')
input = get_input(21)

AA = 'A'
AU = '<v<A>>^A'
AD = '<vA<A>>^A'
AL = '<vA<AA>>^A'
AR = '<vA>^A'

UA = 'vA^A'
UU = 'A'
UD = '<vA>^A'
UL = '<vA<A>>^A'
UR = 'vA<A>^A'

DA = 'vA<^A>A'
DU = '<A>A'
DD = 'A'
DL = '<v<A>>^A'
DR = 'vA^A'

LA = 'vAA<^A>A'
LU = 'vA<^A>A'
LD = 'vA^A'
LL = 'A'
LR = 'vAA^A'

RA = '<A>A'
RU = '<v<A>^A>A'
RD = '<v<A>>^A'
RL = '<v<AA>>^A'
RR = 'A'

MOVES = {
    'AA': AA,
    'A<': AL,
    'A>': AR,
    'A^': AU,
    'Av': AD,
    '^A': UA,
    '^<': UL,
    '^>': UR,
    '^^': UU,
    '^v': UD,
    '<A': LA,
    '<<': LL,
    '<>': LR,
    '<^': LU,
    '<v': LD,
    '>A': RA,
    '><': RL,
    '>>': RR,
    '>^': RU,
    '>v': RD,
    'vA': DA,
    'v<': DL,
    'v>': DR,
    'v^': DU,
    'vv': DD
}
MOVES = {
    'AA': 'A',
    'A<': 'v<<A',
    'A>': 'vA',
    'A^': '<A',
    'Av': 'v<A',
    '^A': '>A',
    '^<': 'v<A',
    '^>': '>vA',
    '^^': 'A',
    '^v': 'vA',
    '<A': '>>^A',
    '<<': 'A',
    '<>': '>>A',
    '<^': '>^A',
    '<v': '>A',
    '>A': '^A',
    '><': '<<A',
    '>>': 'A',
    '>^': '<^A',
    '>v': '<A',
    'vA': '>^A',
    'v<': '<A',
    'v>': '>A',
    'v^': '^A',
    'vv': 'A'
}

'''
029A: <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
        D LL   A RR  U A   L   A R A  D  A   L  U AA R A   L D  AAA R  U A
        R DL   * DR  A *   U   * A *  R  *   D  U ** A *   U D  *** R  A *
               0       *       2   *     3        69   *        63A      *

980A: <v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<vA<A>>^AAAvA<^A>A<vA>^A<A>A\
         L   AAA R A  D LL   A RR  U A  D L   AAA R  U A  D  A U A
         U   *** A *  R DL   * DR  A *  R D   *** R  A *  R  * A *
             369   *         8       *        520      *     A   *

179A: <v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
         L   A  D L   AA RR  U A   L   AA R A  D  AA U A  D L   AAA R  U A
         U   *  D L   ** DR  A *   U   ** A *  R  ** A *  R D   *** R  A *
             3        21       *       47   *     89   *        63A      *
'''

DPAD = {
    '<': LEFT,
    '>': RIGHT,
    '^': UP,
    'v': DOWN
}

class Robot(object):
    def move(self, d):
        #print(self.name, self.pos, d)
        if d == 'A':
            if self.name == 'NUM':
                print(self.pos)
                return self.pos
            else:
                return self.child.move(self.pos)
        else:
            dy, dx = DPAD[d]
            self.x += dx
            self.y += dy
            try:
                self.pos = self.grid[self.y][self.x]
            except:
                print(self.name,d,self.pos)
                raise
            if self.pos == 'X':
                print(self.name,d,self.pos)
                raise
            if self.x < 0 or self.y < 0:
                print(self.name,d,self.pos)
                raise
        
        return ''
    
    
class Dpad(Robot):
    def __init__(self, parent=None, child=None, name='Rob'):
        self.name = name
        self.parent = parent
        self.child = child
        self.pos = 'A'
        self.grid = ["X^A","<v>"]
        self.x = 2
        self.y = 0
    def reset(self):
        self.__init__(self.parent, self.child, self.name)

class Numpad(Robot):
    def __init__(self, parent, name='NUM'):
        self.name = name
        self.child = None
        self.parent = parent
        self.pos = 'A'
        self.grid = ["789", "456", "123", "X0A"]
        self.x = 2
        self.y = 3
    def reset(self):
        self.__init__(self.parent, self.name)
        

def day_twentyone(a, two=False):
    numpad = Numpad()
    dpad2 = Dpad(numpad, 'ROB2')
    dpad1 = Dpad(dpad2, 'ROB1')
    total = 0
    
    robots = []
    if two:
        robots.append(Dpad(numpad, 'ROB25'))
        for i in range(24):
            robots.append(Dpad(robots[-1], f'ROB{24-i}'))
        
        #move = AD+DL+LL+LA+AR+RA
        move = AD+DL+LA+AL+LA+AA+AR+RR+RU+UA
        for m in move:
            robots[-1].move(m)
        for r in robots:print(r.name,r.pos)
    else:
        robots.append(Dpad(numpad, 'ROB2'))
        for i in range(1):
            robots.append(Dpad(robots[-1], f'ROB{1-i}'))
    
        moves = [
            #805A LUUUA, DDDA, UUA, DDRA
            AL + LU + UU + UU + UA + AD + DD + DD + DA + AU + UU + UA + AD + DD + DR + RA,
            #983A UUUA, LA, DDRA, DA
            AU + UU + UU + UA + AL + LA + AD + DD + DR + RA + AD + DA,
            #149A ULLA, UA, URRA, DDDA
            AU + UL + LL + LA + AU + UA + AU + UR + RR + RA + AD + DD + DD + DA,
            #413A UULLA, DA, RRA, DA
            AU + UU + UL + LL + LA + AD + DA + AR + RR + RA + AD + DA,
            #582A LUUA, UA, DDA, DRA
            AL + LU + UU + UA + AU + UA + AD + DD + DA + AD + DR + RA,
        ]
        
    total = 0
    for code, move in zip(a, moves):
        s = ''
        for m in move:
            s += robots[-1].move(m)
        assert code == s, f'{s} != {code}'
        numpad.reset();[r.reset() for r in robots]
        
        total += int(code[:-1]) * len(move)
    
    return total

count = 0
def next_moves(robot, move):
    global count
    pos = robot.pos
    moves = MOVES[pos + move]

    if robot.parent is None:
        for m in moves:
            #robot.move(m)
            count += 1
        return

    for m in moves:
        next_moves(robot.parent, m)

def day_twentyone(a, two=False):
    global count
    numpad = Numpad(None)
    
    moves = [
        #805A LUUUA, DDDA, UUA, DDRA
        '<^^^AvvvA^^Avv>A',
        #983A UUUA, LA, DDRA, DA
        '^^^A<Avv>AvA',
        #149A ULLA, UA, URRA, DDDA
        '^<<A^A^>>AvvvA',
        #413A UULLA, DA, RRA, DA
        '^^<<AvA>>AvA',
        #582A LUUA, UA, DDA, DRA
        '<^^A^AvvAv>A',
    ]
    
    if two:
        N = 18
    else:
        N = 2
    N -= 1 # why?
        
    robots = []
    robots.append(Dpad(None, numpad, f'ROB{N}'))
    for i in range(N):
        r = Dpad(None, robots[-1], f'ROB{1-i}')
        robots[-1].parent = r
        robots.append(r)
        
    total = 0
    for code, moves in zip(a, moves):
        for m in moves:
            next_moves(robots[0], m)
        total += int(code[:-1]) * count
        numpad.reset();[r.reset() for r in robots]
        count = 0
    return total
     
output(day_twentyone(input), ts=1638334897)
#output(day_twentyone(input, True), ts=1638335082)

197541                                                            [00:01:37]


# Day 22

In [299]:
test = """1
10
100
2024""".split('\n')
test2 = """1
2
3
2024""".split('\n')
input = get_input(22)

def get_secret(x):
    x ^= x * 64
    x = x % 16777216
    x ^= x // 32
    x = x % 16777216
    x ^= x * 2048
    return x % 16777216

def day_twentytwo(a, two=False):
    if two:
        seqs = {}
        for x in [int(i) for i in a]:
            seen = set()
            changes = collections.deque([], 4)
            for _ in range(4):
                price = x % 10
                x = get_secret(x)
                changes.append((x%10) - price)
            
            for _ in range(1996):
                price = x % 10
                seq = tuple(changes)
                
                x = get_secret(x)
                changes.append((x%10) - price)
                
                if seq in seen:
                    continue
                
                #print(seq,price)
                seqs[seq] = seqs.get(seq, 0) + price
                seen.add(seq)
        keys = sorted(seqs.keys(), key=lambda k:seqs[k], reverse=True)
        return seqs[keys[0]]
    else:
        total = 0
        for x in [int(i) for i in a]:
            for _ in range(2000):
                x = get_secret(x)
            total += x
        return total

output(day_twentytwo(test), ts=1638334897)
output(day_twentytwo(input), ts=1638334897)
output(day_twentytwo(test2, True), ts=1638335082)
output(day_twentytwo(input, True), ts=1638335082)

37327623                                                          [00:01:37]
16619522798                                                       [00:01:37]
23                                                                [00:04:42]
1854                                                              [00:04:42]


# Day 23

In [476]:
test = """kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn""".split('\n')
input = get_input(23)

def day_twentythree(a, two=False):
    sets = {}
    for row in a:
        cpu1, cpu2 = row.split('-')
        sets[cpu1] = sets.get(cpu1, []) + [cpu2,]
        sets[cpu2] = sets.get(cpu2, []) + [cpu1,]
    
    cpus = sorted(sets.keys())
    num_cpu = len(cpus)
    count = 0
    for i in range(num_cpu-2):
        for j in range(i+1, num_cpu-1):
            for k in range(j+1, num_cpu):
                cpu1 = cpus[i]
                cpu2 = cpus[j]
                cpu3 = cpus[k]
                if cpu1 in sets[cpu2] and cpu2 in sets[cpu3] and cpu3 in sets[cpu1]:
                    if 't' in cpu1[0] + cpu2[0] + cpu3[0]:
                        count += 1
    return count


def day_twentythree(a, two=False):
    g = nx.Graph()
    for row in a:
        cpu1, cpu2 = row.split('-')
        g.add_edge(cpu1, cpu2)
    
    if two:
        biggest = 0
        for clique in nx.find_cliques(g):
            #print(clique)
            if len(clique) > biggest:
                biggest = len(clique)
                party = clique
        return ','.join(sorted(party))
    else:
        count = 0
        seen = set()
        for node in g:
            if not node.startswith('t'):
                continue
            neighbors = list(nx.neighbors(g, node))
            for i in range(len(neighbors)-1):
                for j in range(i+1, len(neighbors)):
                    node2 = neighbors[i]
                    node3 = neighbors[j]
                    if node2 in nx.neighbors(g, node3):
                        lst = tuple(sorted([node,node2,node3]))
                        if lst not in seen:
                            seen.add(lst)
                            count += 1
        return count

output(day_twentythree(test), ts=1638334897)
output(day_twentythree(input), ts=1638334897)
output(day_twentythree(test, True))
output(day_twentythree(input, True))

7                                                                 [00:01:37]
1000                                                              [00:01:37]
co,de,ka,ta
cf,ct,cv,cz,fi,lq,my,pa,sl,tt,vw,wz,yd


# Day 24

In [747]:
test = """x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02""".split('\n')
test2 = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj""".split('\n')
input = get_input(24)

OPS = {
    'OR': '|',
    'XOR': '^',
    'AND': '&'
}

def do_op(op, a, b):
    if op == 'AND':
        return a & b
    elif op == 'OR':
        return a | b
    elif op == 'XOR':
        return a ^ b
    else:
        raise

def day_twentyfour(a, two=False):
    connections = []
    wires = []
    p2 = False
    vals = {}
    for row in a:
        if not row:
            p2 = True
            continue
        elif p2:
            connections.append(row)
            row = row.split()
            wires.append(row[-1])
        else:
            arg, val = row.split()
            vals[arg[:-1]] = int(val)
    count = sum([1 for w in wires if w.startswith('z')])
    
    bits = sorted([k for k in vals if k[0] == 'x'], reverse=True)
    x = vals[bits[0]]
    for shift, b in enumerate(bits[1:]):
        x = x << 1
        x |= vals[b]
    bits = sorted([k for k in vals if k[0] == 'y'], reverse=True)
    y = vals[bits[0]]
    for shift, b in enumerate(bits[1:]):
        y = y << 1
        y |= vals[b]
    
    while True:
        for conn in connections:
            arg1, op, arg2, _, dest = conn.split()
            if dest not in vals:
                if arg1 in vals and arg2 in vals:
                    vals[dest] = do_op(op, vals[arg1], vals[arg2])
        bits = sorted([k for k in vals if k[0] == 'z'], reverse=True)
        if len(bits) == count:
            break

    num = vals[bits[0]]
    for shift, b in enumerate(bits[1:]):
        num = num << 1
        num |= vals[b]
        
    if not two:
        return num
    
    swaps = {
        'z07': 'vmv',
        'vmv': 'z07',
        'z20': 'kfm',
        'kfm': 'z20',
        'z28': 'hnv',
        'hnv': 'z28',
        'hth': 'tqr',
        'tqr': 'hth'
    }
    
    links = {}
    for conn in connections:
        arg1, op, arg2, _, dest = conn.split()
        args = sorted([arg1,arg2])
        if dest in swaps:
            dest = swaps[dest]
        links[f'{args[0]}{op}{args[1]}'] = dest
    
    # Walk through addition logic and bail out if anything breaks
    # Identify failiure manually, and add to swaps above
    assert links['x00XORy00'] == 'z00'
    carry = links['x00ANDy00']
    for i in range(1,len(bits)-1):
        xnode = links[f'x{i:02}XORy{i:02}']
        args = sorted([carry, xnode])
        out = f'z{i:02}'
        key = f'{args[0]}XOR{args[1]}'
        if key not in links or out != links[key]:
            print(i,xnode,out)
            break
        
        carry = links[f'{args[0]}AND{args[1]}']
        anode = links[f'x{i:02}ANDy{i:02}']
        args = sorted([carry, anode])
        if f'{args[0]}OR{args[1]}' not in links:
            print(i,anode)
            break
        
        carry = links[f'{args[0]}OR{args[1]}']
    else:
        return ','.join(sorted(list(swaps.keys())))
    
output(day_twentyfour(test), ts=1638334897)
output(day_twentyfour(test2), ts=1638334897)
output(day_twentyfour(input), ts=1638334897)
output(day_twentyfour(input, True))

4                                                                 [00:01:37]
2024                                                              [00:01:37]
52956035802096                                                    [00:01:37]
hnv,hth,kfm,tqr,vmv,z07,z20,z28


# Day 25

In [748]:
input = get_input(25)

def day_twentyfive(a):
    i = 0
    locks = []
    keys = []
    while i < len(a):
        if not a[i]:
            i += 1
            continue
        
        i += 1
        schema = [0] * 5
        for _ in range(5):
            row = a[i]
            for j in range(5):
                if row[j] == '#':
                    schema[j] += 1
            i += 1
        
        if a[i][0] == '#':
            keys.append(schema)
        else:
            locks.append(schema)
        
        i += 1
    
    total = 0
    for lock in locks:
        for key in keys:
            for i in range(5):
                if lock[i] + key[i] > 5:
                    break
            else:
                total += 1
    return total
        
output(day_twentyfive(input))

3065
